In [1]:
import re
import pickle
import os
from tqdm import tqdm

In [2]:
def lemmaswords(folder):
    all = []
    total_files = sum(len(files) for _, _, files in os.walk(folder))
    with tqdm(total=total_files) as pbar:
        for root, dirs, files in os.walk(folder):
            for file_name in files:
                file_path = os.path.join(root, file_name)
                if os.path.isfile(file_path):
                    if 'ann_morphosyntax.xml' in file_path:
                        with open(file_path, 'r') as f1:
                            word = ''
                            actualp = False
                            preceding = False
                            parenthesis = False
                            for line in f1:
                                if '<seg' in line:
                                    seg = re.search(r'segm_(u-[0-9]*.[0-9]*)', line)
                                if '<f name="orth">' in line:
                                    actualp = True
                                elif actualp:
                                    word = re.search(r'<string>([A-Za-zżźćńółęąśŻŹĆĄŚĘŁÓŃ()]*)<\/string>', line)
                                    actualp = False
                                if '<f name="base">' in line:
                                    preceding = True
                                elif preceding:
                                    lemma = re.search(r'<string>([A-Za-zżźćńółęąśŻŹĆĄŚĘŁÓŃ()]*)<\/string>', line)
                                    if lemma != None:
                                        if lemma.group(1) == '(':
                                            parenthesis = True
                                        elif lemma.group(1) == ')':
                                            parenthesis = False
                                        elif parenthesis == False:
                                            # print(lemma.group(1))
                                            if word != None and seg != None:
                                                all.append((lemma.group(1), word.group(1), seg.group(1), file_path))
                                    preceding = False         
                pbar.update(1)
    return all

In [4]:
with open('C1_lwsf_list.pickle', 'wb') as file:
    pickle.dump(lemmaswords('files/C1/'), file)

100%|██████████| 2538/2538 [09:00<00:00,  4.70it/s]
